In [ ]:
!pip install tensorflow

In [ ]:
pip install opencv-contrib-python

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:

# Preprocessing function
def preprocess_image(image_path, target_size=(128, 128)):
    # 1. Convert to grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 2. Resize image
    img = cv2.resize(img, target_size)

    # 3. Normalize pixel values to [0,1]
    img = img.astype('float32') / 255.0

    # Add channel dimension for CNN
    img = np.expand_dims(img, axis=-1)
    return img

# Load and preprocess dataset
def load_dataset(data_dir, target_size=(128, 128)):
    images = []
    labels = []

    # Define folder paths and corresponding labels
    folders = [
        ('Normal Person ECG Images (284x12=3408)', 0),  # Normal
        ('ECG Images of Myocardial Infarction Patients (240x12=2880)', 1),  # MI
        ('ECG Images of Patient that have History of MI (172x12=2064)', 2),  # History of MI
        ('ECG Images of Patient that have abnormal heartbeat (233x12=2796)', 3)  # Abnormal Heartbeat
    ]

    # Load images from each folder
    for folder_name, label in folders:
        folder_path = os.path.join(data_dir, folder_name)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = preprocess_image(img_path, target_size)
            images.append(img)
            labels.append(label)

    # Convert to arrays and one-hot encode labels
    images = np.array(images)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=4)

    return images, labels



In [ ]:
# Basic CNN Model
def build_cnn_model(input_shape=(128, 128, 1)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(4, activation='softmax')  # Multi-class classification
    ])
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [ ]:
# Convolutional Autoencoder for feature extraction
def build_autoencoder(input_shape=(128, 128, 1)):
    # Encoder
    input_img = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', strides = (1,1), padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', strides = (1,1), padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(16, (3, 3), activation='relu', strides = (1,1), padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', strides = (1,1), padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Encoder model for feature extraction
    encoder = models.Model(input_img, encoded)
    return autoencoder, encoder

In [ ]:
# Classification model using autoencoder features
def build_classifier_from_autoencoder(encoder, input_shape=(32, 32, 16)):
    model = models.Sequential([
        encoder,
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(4, activation='softmax')  # Multi-class classification
    ])
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model


In [ ]:
# Main execution
if __name__ == '__main__':
    # Set dataset directory
    data_dir = '/content/drive/MyDrive/ecg major/major ecg updated/train'
    images, labels = load_dataset(data_dir)

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(
        images, labels, test_size=0.2, random_state=42
    )

    # Train basic CNN
    cnn_model = build_cnn_model()
    cnn_history = cnn_model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_test, y_test)
    )

    # Save CNN model to pickle file
    with open('cnn_model.pkl', 'wb') as f:
        pickle.dump(cnn_model, f)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 817ms/step - accuracy: 0.2637 - loss: 2.1223 - val_accuracy: 0.2796 - val_loss: 1.3823
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 700ms/step - accuracy: 0.2566 - loss: 1.3801 - val_accuracy: 0.2796 - val_loss: 1.3821
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 697ms/step - accuracy: 0.3053 - loss: 1.3764 - val_accuracy: 0.2796 - val_loss: 1.3739
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 697ms/step - accuracy: 0.3008 - loss: 1.3839 - val_accuracy: 0.2796 - val_loss: 1.3761
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 697ms/step - accuracy: 0.3164 - loss: 1.3670 - val_accuracy: 0.2903 - val_loss: 1.3561
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 697ms/step - accuracy: 0.3704 - loss: 1.3202 - val_accuracy: 0.2581 - val_loss: 1.4917
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 677ms/step - accuracy: 0.4286 - loss: 1.2704 - val_accuracy: 0.5591 - val_loss: 1.0824
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 679ms/step - accuracy: 0.5876 - loss: 0.9827 - val_accuracy: 0.655

In [ ]:
# Train autoencoder
autoencoder, encoder = build_autoencoder()
autoencoder.fit(
        X_train, X_train,  # Autoencoder reconstructs input
        epochs=20,
        batch_size=32,
        validation_data=(X_test, X_test)
  )



Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - loss: 0.1226 - val_loss: 0.0226
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0225 - val_loss: 0.0226
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0225 - val_loss: 0.0226
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0226 - val_loss: 0.0226
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0226 - val_loss: 0.0226
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0225 - val_loss: 0.0226
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0225 - val_loss: 0.0226
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0226 - val_loss: 0.0226
Epoch 9/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - loss: 0.0224 - val_loss: 0.0226
Epoch 10/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 0.0227 - val_loss: 0.0226
Epoch 11/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - loss: 0.0226 - val_loss: 0.0226
Epoch 12/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - loss: 0.0224 - val_lo

In [ ]:
# Train classifier with autoencoder features
classifier = build_classifier_from_autoencoder(encoder)
classifier.fit(
        X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_test, y_test)
)


Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 457ms/step - accuracy: 0.2915 - loss: 4.4214 - val_accuracy: 0.1828 - val_loss: 1.3869
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 440ms/step - accuracy: 0.2290 - loss: 1.4203 - val_accuracy: 0.2581 - val_loss: 1.3859
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 432ms/step - accuracy: 0.2523 - loss: 1.3856 - val_accuracy: 0.2796 - val_loss: 1.3849
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 427ms/step - accuracy: 0.2783 - loss: 1.3844 - val_accuracy: 0.2796 - val_loss: 1.3838
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 421ms/step - accuracy: 0.3183 - loss: 1.3829 - val_accuracy: 0.2796 - val_loss: 1.3827
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 431ms/step - accuracy: 0.3310 - loss: 1.3804 - val_accuracy: 0.2796 - val_loss: 1.3817
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 436ms/step - accuracy: 0.3199 - loss: 1.3799 - val_accuracy: 0.2796 - val_loss: 1.3808
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 423ms/step - accuracy: 0.3258 - loss: 1.3791 - val_accu

In [ ]:
 # Train tuned classifier with autoencoder features
classifier1 = build_classifier_from_autoencoder(encoder)
classifier1.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(X_test, y_test)
    )

    # Save tuned classifier to pickle file
with open('classifier1.pkl', 'wb') as f:
  pickle.dump(classifier1, f)

Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 453ms/step - accuracy: 0.2952 - loss: 3.4432 - val_accuracy: 0.4355 - val_loss: 1.1904
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 431ms/step - accuracy: 0.5621 - loss: 1.1016 - val_accuracy: 0.7043 - val_loss: 0.9824
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 419ms/step - accuracy: 0.7305 - loss: 0.8697 - val_accuracy: 0.7258 - val_loss: 0.7633
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 422ms/step - accuracy: 0.8250 - loss: 0.5762 - val_accuracy: 0.8011 - val_loss: 0.5683
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 419ms/step - accuracy: 0.8868 - loss: 0.4317 - val_accuracy: 0.8387 - val_loss: 0.4767
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 418ms/step - accuracy: 0.8941 - loss: 0.3439 - val_accuracy: 0.8871 - val_loss: 0.3665
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 417ms/step - accuracy: 0.9442 - loss: 0.2340 - val_accuracy: 0.8817 - val_loss: 0.3541
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 423ms/step - accuracy: 0.9707 - loss: 0.1820 - val_accu

In [7]:
# @title Default title text
print("Basic CNN Test Accuracy:", f"{cnn_score[1]:.4f}")
print("Improved Convolutional Autoencoder Test Accuracy:", f"{ae_score[1]:.4f}")
print("Tuned Convolutional Autoencoder Test Accuracy:", f"{tuned_ae_score[1]:.4f}")

Basic CNN Test Accuracy: 0.8541
Improved Convolutional Autoencoder Test Accuracy: 0.8750
Tuned Convolutional Autoencoder Test Accuracy: 0.9255
